# Celebrity Screen Time Calculator

---



In [1]:
# Required packages for running this program

!pip install face_recognition opencv-python numpy sklearn

     |████████████████████████████████| 100.2MB 63kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=716c009a72ac25889414548f52d5c94c539244adb2f45bbf70ddc8e17bce8eea
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [2]:
# Required imports

import cv2
import face_recognition
import numpy
import time
from google.colab.patches import cv2_imshow
from sklearn import svm
import os

In [ ]:
# Train multiple images per person
# Find and recognize faces in an image using a SVC with scikit-learn

# The training data would be all the face encodings from all the known images and the labels are their names
encodings = []
names = []

# Training directory
train_dir = os.listdir('/content/drive/My Drive/data/train/')
train_directory = '/content/drive/My Drive/data/train/'

# Loop through each person in the training directory
for person in train_dir:

	personPictures = os.listdir(train_directory + person)

	# Loop through each training image for the current person
	for person_img in personPictures:

		# Get the face encodings for the face in each image file
		face = face_recognition.load_image_file(train_directory + person + "/" + person_img)
		face_bounding_boxes = face_recognition.face_locations(face)

		#If training image contains exactly one face
		if len(face_bounding_boxes) == 1:
			face_enc = face_recognition.face_encodings(face)[0]
			# Add face encoding for current image with corresponding label (name) to the training data
			encodings.append(face_enc)
			names.append(person)
		else:
			print(person + "/" + person_img + " was skipped and can't be used for training")

print("Training Complete.")

Training data format:

*   Train/
    *   Celebrity1/
        * Celebrity1image1.jpg
        * Celebrity1image2.jpg
        * Celebrity1image3.jpg
    *   Celebrity2.
        * Celebrity2image1.jpg
        * Celebrity2image2.jpg
        * Celebrity2image3.jpg
    *   Celebrity3.
        * Celebrity2image1.jpg
        * Celebrity2image2.jpg
        * Celebrity2image3.jpg



In [4]:
# Create and train the SVC classifier

clf = svm.SVC(gamma='scale')
clf.fit(encodings, names) #encodings are from the image, names are the labels for these encodings

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Use this classifier to detect which celebrity is on screen.

In [13]:
from google.colab.patches import cv2_imshow
import math

# import the video file
capture = cv2.VideoCapture('/content/drive/My Drive/The.Office.S03E02.mkv') # Uplaod your video file here

# initialize variables 
frames_Micheal_is_in = 0
current_frame = 0
process_This = True
resize_factor = 0.25

# Calculate total frames, fps and duration of video in minutes
total_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
fps = capture.get(cv2.CAP_PROP_FPS)     
duration = total_frames/(fps*60)

# 
print("Width:{} x Height:{}".format(int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Width and height of video with resize factor
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)) * resize_factor 
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT)) * resize_factor

print("Resized Width:{} x Height:{}".format(int(width), int(height)))

Width:1280 x Height:720
Resized Width:320 x Height:180


In [14]:
t0 = time.time()

while capture.isOpened():
    ret, frame = capture.read()
    if ret:
        current_frame += 1
        if process_This:
            frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)

            # Finding all the faces on this frame and storing their encodings
            face_encodings = face_recognition.face_encodings(frame)

            for face_encoding in face_encodings:

                # Checking each face encoding with our classifier for matches
                name = clf.predict([face_encoding])

                #If the output label is Steve Carrell, He is in the frame
                if name == 'Steve Carrell':
                    #cv2_imshow(frame)  # This displays all the frames Steve Carrell is in
                    frames_Micheal_is_in += 1
            
        if current_frame % (total_frames//10) == 0:
            print("{}% video processed.".format(math.ceil((current_frame/total_frames) * 100)))

        process_This = not process_This

    if current_frame == total_frames:
        break

print("Time taken to process video of resolution {}x{} = {:.2f} minutes".format(int(width), int(height), ((time.time() - t0)/ 60)))
print("Duration of episode is {:.2f} minutes. Micheal Scott (Steve Carrell) is on screen for {:.2f} minutes.".format(duration, ((2*frames_Micheal_is_in)/(fps*60))))

10% video processed.
20% video processed.
30% video processed.
40% video processed.
50% video processed.
60% video processed.
70% video processed.
80% video processed.
90% video processed.
100% video processed.
Time taken to process video of resolution 320x180 = 14.59 minutes
Duration of episode is 21.46 minutes. Micheal Scott (Steve Carrell) is on screen for 6.12 minutes.
